# XGBoost Benchmark

In [0]:
%pip install --extra-index-url https://pypi.nvidia.com cudf-cu11
%pip install cuml-cu11 --extra-index-url https://pypi.nvidia.com

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.3/489.3 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 79.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 74.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.8/409.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.3/89.3 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.1/553.1 kB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 70.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [0]:
%pip install openpyxl
%pip install scikit-image
%pip install dask-ml
dbutils.library.restartPython()

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 5.8 MB/s eta 0:00:00
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 36.8 MB/s eta 0:00:00
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.7/148.7 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 36.0 MB/s eta 0:

In [0]:
import pandas as pd
from sklearn.model_selection import (
    RandomizedSearchCV,
    train_test_split,
    StratifiedKFold,
)
from sklearn.pipeline import Pipeline
#from sklearn.feature_extraction.text import TfidfVectorizer
from cuml.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import (
    confusion_matrix,
    ConfusionMatrixDisplay,
    precision_recall_fscore_support,
    f1_score,
    recall_score,
    precision_score,
)
from xgboost import XGBClassifier
import xgboost as xgb
import scipy.stats as st
import mlflow
from mlflow.tracking import MlflowClient
import matplotlib.pyplot as plt
from mlflow.utils.environment import _mlflow_conda_env
from mlflow.models.signature import infer_signature
import matplotlib
import io
from PIL import Image
import numpy as np
import seaborn as sns
import sklearn
import cuml
import cudf

import dask
import dask.dataframe as dd
from dask.distributed import Client, wait, progress, get_worker
from xgboost.dask import DaskDMatrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import compute_sample_weight


print('xgboost version', xgb.__version__)

run_type = 'cpu'
max_depth = 15
num_boost_round = 200
score_metric = 'auc'
storage_backend = 'local'

DATA_PATH = '/dbfs/FileStore/tables/processed_data_june_dask.parquet'
included_features = ['TAGS', 'ACTION', 'OBJECT', 'PRIOR_ACTIONS', 'ROOT CAUSE', 'screenshot', 'IssueType', 'CAT']
label_col = 'CAT'
xgb_model_name = 'xgb_'+run_type+'_autoresult_analysis_max_depth_'+str(max_depth)

if run_type == 'gpu':
    from dask_cuda import LocalCUDACluster
    
    # Run on all available GPU on same machine:
    cluster = LocalCUDACluster(threads_per_worker=16, memory_limit='128GB')
    client = Client(cluster)
    
    # Run single GPU:
#     client = Client(n_workers=1, threads_per_worker=32)
elif run_type == 'cpu':
    client = Client(n_workers=1, threads_per_worker=32)
#     client = Client(n_workers=2, threads_per_worker=16)   
    
client

xgboost version 1.7.6


/local_disk0/.ephemeral_nfs/envs/pythonEnv-fa9614d8-7e9c-40b7-9e5e-09b1f38d56f7/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37865 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:37865/status,
Dashboard: http://127.0.0.1:37865/status,Workers: 1
Total threads: 32,Total memory: 10.83 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44681,Workers: 1
Dashboard: http://127.0.0.1:37865/status,Total threads: 32
Started: Just now,Total memory: 10.83 GiB
Comm: tcp://127.0.0.1:45443,Total threads: 32
Dashboard: http://127.0.0.1:40381/status,Memory: 10.83 GiB
Nanny: tcp://127.0.0.1:42759,


# Prepare Data

In [0]:
PATH = "/dbfs/mnt/s3_mount/qe_automation/data/"
aws_bucket_name = "ml-hub-workspace-root-sg50"
mount_name = "s3_mount"
mount_point = "/mnt/%s" % mount_name
try:
    dbutils.fs.mount("s3a://%s" % aws_bucket_name, mount_point)
except Exception as e:
    print("Already mounted!")
#dbutils.fs.mount(f"s3a://{aws_bucket_name}", f"/mnt/{mount_name}")
display(dbutils.fs.ls(f"/mnt/{mount_name}"))

Already mounted!


path,name,size,modificationTime
dbfs:/mnt/s3_mount/nvirginia-prod/,nvirginia-prod/,0,1691165785481
dbfs:/mnt/s3_mount/qe_automation/,qe_automation/,0,1691165785481
dbfs:/mnt/s3_mount/weights/,weights/,0,1691165785481


In [0]:
ls /dbfs/mnt/s3_mount/

nvirginia-prod/  qe_automation/  weights/


In [0]:
import pandas as pd

encode_dict = {}
def encode_label(x):
    if x not in encode_dict.keys():
        encode_dict[x] = len(encode_dict)
    return encode_dict[x]

#df = pd.read_csv('/dbfs/FileStore/shared_uploads/Chris.Jose@cvent.com/subset.csv')
df = dd.read_parquet('/dbfs/FileStore/tables/data_optimized_proc.parquet')
df["CAT"] = df["IssueType"].map(lambda x: encode_label(x))
df

,ExecutionItemID,StepLogID,IssueType,TestCaseStatus,StepDescription,screenshot,ExecutionId,image_id,CAT
npartitions=1,,,,,,,,,
,int64,int64,object,object,object,object,int64,object,int64
,...,...,...,...,...,...,...,...,...


In [0]:
dd.iloc[22]

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File <command-4152036570402745>, line 1
----> 1 dd.iloc[22]

AttributeError: module 'dask.dataframe' has no attribute 'iloc'

In [0]:
import numpy as np
import re
keys = ['TAGS', 'ACTION', 'OBJECT', 'TEST DATA', 'DB DATA', 'PRIOR_ACTIONS', 'ROOT CAUSE'] #'PLATFORM', 'CAPABILITIES', 'RESOLUTION']

def extract_values(text, key):
    pattern = re.compile(rf'{key}:\s*(.*?)(?:\n|$|▐)')
    match = pattern.search(text)
    return match.group(1).strip() if match else None

def split_text(keys, data):
    result = {k: [] for k in keys}
    for idx, ds in enumerate(data["StepDescription"]):
        for key in keys:
            result[key].append(extract_values(ds, key))
    return pd.DataFrame(result)

def split_text_1(data):
    result = {k: [] for k in keys}
    print(data.columns)
    for idx, ds in enumerate(data["StepDescription"]):
        for line in ds.split('\n'):
            split_div = line.split('▐')
            if len(split_div) > 1:
                for s in split_div:
                    parts = s.split(':')
                    k = parts[0].strip()
                    if k in keys:
                        v = parts[1].strip()
                        result[k].append(v)
                    if k== 'UTILITY ACTION':
                        result['ACTION'].append(parts[1].strip())

            else:
                parts = line.split(':')
                k = parts[0].strip()
                if k in keys:
                    v = ''.join(parts[1:]).strip()
                    result[k].append(v)
        for k in keys:
            if len(result[k]) < idx + 1:
                result[k].append(np.nan)
    return pd.DataFrame(result)

#print(split_text(df.StepDescription[1]))

parsed_ds = split_text(keys, df)
subset = df.drop(columns=['StepDescription'])
print("PARSED: ", parsed_ds.shape[0])
print("SUBSET: ", subset.shape[0])
#subset['TAGS'] = parsed_ds['TAGS']
parsed_ds = parsed_ds.reset_index(drop=True)
subset = subset.reset_index(drop=True)
final_df = pd.concat([subset, parsed_ds], axis=1)
#final_df.reset_index(inplace=True)
final_df.to_parquet('/dbfs/FileStore/tables/processed_data_june_dask.parquet')
#final_df = final_df[final_df['ExecutionItemID'].isna()]
#final_df
final_df

PARSED:  17800
SUBSET:  Delayed('int-6820ab00-cf34-4be1-98b1-efbb0b398d63')


---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File <command-2862209793706248>, line 53
     51 parsed_ds = parsed_ds.reset_index(drop=True)
     52 subset = subset.reset_index(drop=True)
---> 53 final_df = pd.concat([subset, parsed_ds], axis=1)
     54 #final_df.reset_index(inplace=True)
     55 final_df.to_parquet('/dbfs/FileStore/tables/processed_data_june_dask.parquet')

File /databricks/python/lib/python3.10/site-packages/pandas/util/_decorators.py:311, in deprecate_nonkeyword_arguments.<locals>.decorate.<locals>.wrapper(*args, **kwargs)
    305 if len(args) > num_allow_args:
    306     warnings.warn(
    307         msg.format(arguments=arguments),
    308         FutureWarning,
    309         stacklevel=stacklevel,
    310     )
--> 311 return func(*args, **kwargs)

File /databricks/python/lib/python3.10/site-packages/pandas/core/reshape/concat.py:347, in concat(objs, axis,

In [0]:
if run_type == 'cpu':
     ddf_enc = dd.read_parquet(DATA_PATH)
elif run_type == 'gpu':
    import dask_cudf
    import cupy as cp
    ddf_enc = dask_cudf.read_parquet(DATA_PATH)
else:
    raise ValueError('Select run_type of cpu or gpu.')
#df = df.drop(columns=['RESOLUTION'])
ddf_enc = ddf_enc[['TAGS', 'ACTION', 'OBJECT', 'PRIOR_ACTIONS', 'ROOT CAUSE', 'screenshot', 'IssueType', 'CAT']]
ddf_enc = ddf_enc[ddf_enc['IssueType'] != 'PROCESSING_ISSUE'] 
ddf_enc = ddf_enc.dropna()
# Weighted Random Sampler for class imbalance
ddf_enc['class_weight'] = compute_sample_weight('balanced', ddf_enc[label_col]))
ddf_enc

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
File <command-1011379387708872>, line 14
     12 ddf_enc = ddf_enc.dropna()
     13 # Weighted Random Sampler for class imbalance
---> 14 ddf_enc['class_weight'] = pd.Series(compute_sample_weight('balanced', ddf_enc[label_col]))
     15 ddf_enc

File /local_disk0/.ephemeral_nfs/envs/pythonEnv-fa9614d8-7e9c-40b7-9e5e-09b1f38d56f7/lib/python3.10/site-packages/dask/dataframe/core.py:4864, in DataFrame.__setitem__(self, key, value)
   4862     raise NotImplementedError(f"Item assignment with {type(key)} not supported")
   4863 else:
-> 4864     df = self.assign(**{key: value})
   4866 self.dask = df.dask
   4867 self._name = df._name

File /local_disk0/.ephemeral_nfs/envs/pythonEnv-fa9614d8-7e9c-40b7-9e5e-09b1f38d56f7/lib/python3.10/site-packages/dask/dataframe/core.py:5305, in DataFrame.assign(self, **kwargs)
   5301     # Figure out colum

In [0]:
ddf_enc.ACTION.value_counts()

Dask Series Structure:
npartitions=1
    int64
      ...
Name: ACTION, dtype: int64
Dask Name: value-counts-agg, 9 graph layers

In [0]:
#Label encode actions/prior actions
from sklearn.preprocessing import LabelEncoder
#label_enc = LabelEncoder()
#df['ACTION_Enc'] = label_enc.fit_transform(df["ACTION"])
#df['PRIOR_ACTIONS_SPLIT'] = df['PRIOR_ACTIONS'].apply(lambda entry: entry.split('|'))
ddf_enc.head()

,TAGS,ACTION,OBJECT,PRIOR_ACTIONS,ROOT CAUSE,screenshot,IssueType,CAT
0,@us @regression @csn @DirectBook @DirectBook_B...,Input,DirectBook.Booking-GuestInfo.btnGuestName,Input | Input | Input | Input | Input,WaitTimeoutException: TIMED OUT AFTER 30 TOTAL...,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,Sync Issue,0
1,@us @regression @csn @DirectBook @DirectBook_B...,Input,DirectBook.Booking-VnuRoomDetails.lnkFirstRoom...,Input | Input | VerifyExistence | Input | Wait...,WaitTimeoutException: TIMED OUT AFTER 1 TOTAL ...,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,Sync Issue,0
2,@us @flex @regression @session_bundles @wax @a...,VerifyExistence,Flex.Guest_Side_Summary.txt-congratulations-co...,Input | VerifyExistence | VerifyExistence | In...,WaitTimeoutException: TIMED OUT AFTER 1 TOTAL ...,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,Sync Issue,0
3,@us @flex @regression @session_bundles @wax @a...,VerifyExistence,Flex.Guest_Side_Registration.txtRegSummaryPage...,WaitForSeconds | VerifyExistence | VerifyExist...,HardErrorException: Application Issue Found wi...,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,Sync Issue,0
5,@us @regression @csn @DirectBook @DirectBook_B...,Output,DirectBook.Booking-LandingPage.txtDropDownName,Input | | Input | Input | Output,WaitTimeoutException: TIMED OUT AFTER 1 TOTAL ...,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,Sync Issue,0


In [0]:
from sklearn.model_selection import train_test_split
df['class_weight'] = compute_sample_weight('balanced', df[label_col])
train_dataset, test_dataset = train_test_split(
    ddf_enc, train_size=0.8, random_state=42, stratify=df.CAT
)
train_dataset = train_dataset.reset_index(drop=True)
# train_dataset = train_dataset[train_dataset['IssueType'] != "SYNC_ISSUE"]
test_dataset = test_dataset.reset_index(drop=True)
print("- TRAIN -\n")
print(train_dataset.groupby(["CAT", "IssueType"])["CAT"].count())
print("-----------------")
print("- TEST -\n")
print(test_dataset.groupby(["CAT", "IssueType"])["CAT"].count())

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-1713639788819841>, line 2
      1 from sklearn.model_selection import train_test_split
----> 2 df['class_weight'] = compute_sample_weight('balanced', df[label_col])
      3 train_dataset, test_dataset = train_test_split(
      4     ddf_enc, train_size=0.8, random_state=42, stratify=df.CAT
      5 )
      6 train_dataset = train_dataset.reset_index(drop=True)

NameError: name 'df' is not defined

# Set CV Params

Hyperparam ranges cloned from Job Classifier

In [0]:
def get_params():
    learning_rate = st.uniform(0.05, 0.4)
    max_depth = [10, 20, 50, 100, 200, 300, 500]
    n_estimators = [5, 10, 25, 50, 75, 100, 200, 400]
    reg_lambda = [0.1, 0.3, 1]
    colsample_bytree = st.beta(10, 1)
    gamma = st.uniform(0, 10)
    subsample = [0.6, 0.8, 1.0]
    reg_alpha = st.expon(0, 50)
    min_child_weight = [1, 5, 10]
    # Create the random grid
    return {
        "vectorizer__analyzer": ["word"],
        "vectorizer__stop_words": ["english"],
        "vectorizer__strip_accents": ["ascii"],
        "vectorizer__ngram_range": [(1, 1), (1, 2), (2, 2)],
        "classifier__n_estimators": n_estimators,
        "classifier__learning_rate": learning_rate,
        "classifier__reg_lambda": reg_lambda,
        "classifier__colsample_bytree": colsample_bytree,
        "classifier__gamma": gamma,
        "classifier__subsample": subsample,
        "classifier__reg_alpha": reg_alpha,
        "classifier__max_depth": max_depth,
        "classifier__min_child_weight": min_child_weight,
    }

# Define model

In [0]:
def dask_xgb_train(client, run_type, ddf_enc, label_col, xgb_params, xgb_model_name):
    """
    Use dask + xgboost for training against already encoded data.
    """
    tic = time()
    ddf_enc = client.persist(ddf_enc)
#     wait([ddf_enc])

    feature_cols = [cc for cc in ddf_enc.columns if cc in included_features]
    print('Feature columns:', feature_cols)
    print('  Number of feature columns:', len(feature_cols))

   # preprocessor = ColumnTransformer(
   #  transformers=[
   #      ('text', TfidfVectorizer(), 'raw_text_ft'), #TfidfVectorizer accepts column name only between quotes
   #      ('category', OneHotEncoder(), ['categorical_ft']),
   #  ],
   # )

    vectorizer = CountVectorizer()
    encoder = OneHotEncoder()
    mlb = MultiLabelBinarizer()
    X_text = vectorizer.fit_transform(ddf_enc['TAGS'] + ' ' + ddf_enc['OBJECT'] + ' ' + ddf_enc['ROOT CAUSE']).toarray()
    X_ohe = encoder.fit_transform(ddf_enc[["ACTION"]]).toarray()
    X_prior_actions = mlb.fit_transform(ddf_enc['PRIOR_ACTIONS'].split('|')).toarray()
    X = np.hstack((X_test, X_ohe, X_prior_actions))

    #X = ddf_enc[feature_cols].astype('float32')
    y = ddf_enc[label_col].astype('float32')
    
    # Per-instance weight required due to severe class imbalance in One-vs-Rest decomposition.
    # Weights precalculated during ETL on training set alone to avoid leakage.
    weight = ddf_enc['class_weight'].astype('float32')
    
#     # Persist data:
#     X = client.persist(X)
#     y = client.persist(y)
#     weight = client.persist(weight)
#     wait([X, y, weight])

    # Use XGBOOST API:
    if run_type == 'gpu':
        # See example: https://github.com/dmlc/xgboost/blob/master/demo/dask/gpu_training.py
        # `DaskDeviceQuantileDMatrix` is used instead of `DaskDMatrix`, be careful
        # that it can not be used for anything else than training.
        dtrain = xgb.dask.DaskDeviceQuantileDMatrix(client, X, y, weight=weight)
    else:
        dtrain = DaskDMatrix(client, X, y, weight=weight)
    
#     del ddf_enc, X, y # Cleanup
    data_load_time = np.round(time() - tic, 2)

    tic = time()
    xgb_model = xgb.dask.train(client, xgb_params, dtrain, num_boost_round=num_boost_round)['booster']
    train_model_time = np.round(time() - tic, 2)

    # Save xgb model to disk:
    tic = time()
    xgb_model.save_model(xgb_model_name+'.model') # Save model to current dir
    xgb_model.save_model('/tmp/'+xgb_model_name+'.json') # Experimental JSON format. To be nested in output file with timings/metrics.

    # Move local xgb-model to hdfs if xgb_hdfs_output_dir specified:
    if (storage_backend == 'hdfs'):
        if xgb_hdfs_output_dir != None:
            with open(xgb_model_name+'.model','rb') as f:
                hdfs.upload(os.path.join(xgb_hdfs_output_dir, xgb_model_name+'.model'), f)
    export_model_time = np.round(time() - tic, 2)

    metrics = {
        'data_load_time': data_load_time,
        'train_model_time': train_model_time,
        'export_model_time': export_model_time
        }

    del dtrain
    gc.collect() # Manual garbage collection needed since python may not clean up GPU resources automatically.
    return(metrics)


# xgboost params:
xgb_params = { 
    'random_state': 0,
    'eta': 0.1,
    'gamma': 0.1,
    'max_depth': max_depth, 
#     'num_boost_round': num_boost_round, # Number of boosting rounds: num_boost_round in xgboost, numRound in xgboost4j, n_estimators in scikit-learn.
    'max_leaves': 4*256, # Maximum number of nodes to be added. Only relevant when grow_policy=lossguide is set.
    'objective': xgb_objective,
    'eval_metric': 'auc', # Applied to eval_set/test_data if provided
    'booster': 'gbtree',
}

if run_type == 'cpu':
    tree_method = 'hist'
elif run_type == 'gpu':   
    tree_method = 'gpu_hist'

xgb_params['tree_method'] = tree_method


y_tmp = dd.read_parquet('/dbfs/FileStore/tables/processed_data_june.parquet', columns=label_col)
num_class = y_tmp.max().compute() + 1

print('Number of classes detected: ', num_class)
xgb_params['num_class'] = num_class

print('Starting xgboost '+run_type+' '+pred_model+' training....')

tic = time()

# Large amount of memory required to train on full 16 yrs of data. Reduce number of years used during training on single node tests.
training_metrics = dask_xgb_train_from_enc(client, run_type, train_dataset, label_col, xgb_params, xgb_model_name)

t_dask_train = np.round(time() - tic, 2)

training_metrics['total_time'] = t_dask_train
print(training_metrics)
print('Dask xgboost '+run_type+' training time:', '{:0.2f}'.format(t_dask_train) + 's')



---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-1049894031074854>, line 88
     77     return(metrics)
     80 # xgboost params:
     81 xgb_params = { 
     82     'random_state': 0,
     83     'eta': 0.1,
     84     'gamma': 0.1,
     85     'max_depth': max_depth, 
     86 #     'num_boost_round': num_boost_round, # Number of boosting rounds: num_boost_round in xgboost, numRound in xgboost4j, n_estimators in scikit-learn.
     87     'max_leaves': 4*256, # Maximum number of nodes to be added. Only relevant when grow_policy=lossguide is set.
---> 88     'objective': xgb_objective,
     89     'eval_metric': 'auc', # Applied to eval_set/test_data if provided
     90     'booster': 'gbtree',
     91 }
     93 if run_type == 'cpu':
     94     tree_method = 'hist'

NameError: name 'xgb_objective' is not defined

In [0]:
from sklearn.preprocessing import LabelEncoder
keys = ['TAGS', 'ACTION', 'OBJECT','PRIOR_ACTIONS', 'ROOT CAUSE'] #'PLATFORM', 'CAPABILITIES', 'RESOLUTION']

def train_tree_model(
    train_data: pd.DataFrame, cv_params: dict, n_cv_folds: int, n_iter: int
):
    """
    :param train_data: Preprocessed dataframe that contains both the question and category
    """
    columns =['TAGS', 'ACTION', 'OBJECT', 'PRIOR_ACTIONS', 'ROOT CAUSE']
   # tfidf = TfidfVectorzer()
   # for feature in columns:
    #    df[feature + '_tfidf'] = tfidf.fit_transform(train_data[feature]).toarray()

    #X = pd.concat([train_data[feature + '_tfidf'] for feature in train_data.columns], axis=1)
    X = train_data[keys]
    
    le = LabelEncoder()
    Y = le.fit_transform(train_data["CAT"])
    #Y = train_data["CAT"]

   # feature_union = FeatureUnion([
   #     ('tfidf1', T)
   # ])

    classifier = Pipeline(
        steps=[
            ('vectorizer', TfidfVectorizer()),
            ("classifier", XGBClassifier(random_state=10)),
        ],
        verbose=True,
    )
    classifier_random = RandomizedSearchCV(
        classifier,
        param_distributions=cv_params,
        n_iter=n_iter,
        verbose=0,
        scoring="accuracy",
        random_state=42,
        refit=True,
        n_jobs=-1,
        cv=StratifiedKFold(n_splits=n_cv_folds, shuffle=True, random_state=42),
    )
    classifier_random.fit(X, Y)
    return classifier_random

## Model storage function

In [0]:
class SKLearnEstimatorWrapper(mlflow.pyfunc.PythonModel):
    def __init__(self, estimator, mapping_dict):
        self.estimator = estimator
        self.mapping_dict = mapping_dict

    def predict(self, context, model_input):
        #image_features = np.array([hog(np.array(Image.open(io.BytesIO(model_input.loc[i, "screenshot"]))), channel_axis=-1) for i in range(len(model_input))]) 

        model_input = model_input[keys]#, 'image': image_features}

        outputs = self.estimator.predict(model_input)
        ret_d = {"class": np.array([self.mapping_dict[str(x)] for x in outputs])}
        return ret_d


def log_estimator_model(estimator, encode_dict):
    """
    :param estimator: An estimator that takes as input text and outputs a category (for the question)
    :param encode_dict: A dictionary which encodes a labelled category (such as 'Guest Rooms') to a numeric category
    """
    mapping_dict = {str(v): k for k, v in encode_dict.items()}
    wrapped_estimator = SKLearnEstimatorWrapper(estimator, mapping_dict)
    input_example = {
        "text": np.array(
            test_dataset.loc[0, keys]
        )
        #'image': hog(np.array(Image.open(io.BytesIO(test_dataset.loc[0, "screenshot"]))), channel_axis=-1)
    }
    signature = infer_signature(
        pd.DataFrame(input_example),
        wrapped_estimator.predict(None, pd.DataFrame(input_example)),
    )
    # Specify the additional dependencies
    conda_env = _mlflow_conda_env(
        additional_conda_deps=None,
        additional_pip_deps=[
            "xgboost=={}".format(xgboost.__version__),
            "sklearn=={}".format(sklearn.__version__),
        ],
        additional_conda_channels=None,
    )
    # Log model so the model artifacts can have the correct conda env, python model, and signature (data formats for input & output)
    return mlflow.pyfunc.log_model(
        "Bluecumber_test_analysis",
        python_model=wrapped_estimator,
        conda_env=conda_env,
        signature=signature,
        input_example=input_example,
    )

# Perform CV with MlFlow

In [0]:
client = MlflowClient()
n_cv_folds = 3


def init_experiment(experiment_name):
    # fetch existing experiments, if it doesn't exist create it and set it
    existing_experiments = client.search_experiments()
    if experiment_name in [exp.name for exp in existing_experiments]:
        return mlflow.set_experiment(experiment_name)
    else:
        return mlflow.create_experiment(experiment_name)

In [0]:
keys = ['TAGS', 'ACTION', 'OBJECT', 'PRIOR_ACTIONS', 'ROOT CAUSE'] #'PLATFORM', 'CAPABILITIES', 'RESOLUTION']


mlflow.end_run()
#experiment_id = init_experiment(
  #  "/Users/Chris.Jose@cvent.com/test_result_analysis/experiments/#xgboost_test_result_analysis_1")
#run = mlflow.start_run(run_name="xgboost_tfidf_CV", #experiment_id=experiment_id)

experiment_name = "/Users/Chris.Jose@cvent.com/test_result_analysis/experiments/#xgboost_test_result_analysis_2"
mlflow.set_experiment(experiment_name)  
  
# Get the experiment ID  
experiment = mlflow.get_experiment_by_name(experiment_name)  
experiment_id = experiment.experiment_id  
  
# Start a new MLflow run  
run = mlflow.start_run(run_name="xgboost_tfidf_CV", experiment_id=experiment_id)


mlflow.sklearn.autolog(log_models=False)
mlflow.log_param("n_cv_folds", 3)
classifier_random = train_tree_model(
    train_dataset, get_params(), n_cv_folds, n_iter=30000
)
print(classifier_random.best_score_)
#test_image_features = np.array([hog(np.array(Image.open(io.BytesIO(test_dataset.loc[i, "screenshot"]))), channel_axis=-1) for i in range(len(test_dataset))]) 

#model_input = {'text': test_dataset["StepDescription"], 'image': test_image_features}
X_test = test_dataset[keys]
Y_test = test_dataset["CAT"]
mlflow.log_metric(
    "testing_accuracy_score", classifier_random.best_estimator_.score(X_test, Y_test)
)
estimator = classifier_random.best_estimator_

com.databricks.rpc.RPCResponseTooLarge: rpc response (of 20985024 bytes) exceeds limit of 20971520 bytes
	at com.databricks.rpc.Jetty9Client$$anon$1.onContent(Jetty9Client.scala:651)
	at shaded.v9_4.org.eclipse.jetty.client.api.Response$Listener.onContent(Response.java:294)
	at shaded.v9_4.org.eclipse.jetty.client.api.Response$Listener.onContent(Response.java:306)
	at shaded.v9_4.org.eclipse.jetty.client.ResponseNotifier.notifyContent(ResponseNotifier.java:155)
	at shaded.v9_4.org.eclipse.jetty.client.ResponseNotifier.notifyContent(ResponseNotifier.java:139)
	at shaded.v9_4.org.eclipse.jetty.client.HttpReceiver$ContentListeners.notifyContent(HttpReceiver.java:725)
	at shaded.v9_4.org.eclipse.jetty.client.HttpReceiver$ContentListeners.access$500(HttpReceiver.java:688)
	at shaded.v9_4.org.eclipse.jetty.client.HttpReceiver.responseContent(HttpReceiver.java:409)
	at shaded.v9_4.org.eclipse.jetty.client.http.HttpReceiverOverHTTP.content(HttpReceiverOverHTTP.java:295)
	at shaded.v9_4.org.eclipse.jetty.http.HttpParser.parseContent(HttpParser.java:1815)
	at shaded.v9_4.org.eclipse.jetty.http.HttpParser.parseNext(HttpParser.java:1526)
	at shaded.v9_4.org.eclipse.jetty.client.http.HttpReceiverOverHTTP.parse(HttpReceiverOverHTTP.java:200)
	at shaded.v9_4.org.eclipse.jetty.client.http.HttpReceiverOverHTTP.process(HttpReceiverOverHTTP.java:141)
	at shaded.v9_4.org.eclipse.jetty.client.http.HttpReceiverOverHTTP.receive(HttpReceiverOverHTTP.java:75)
	at shaded.v9_4.org.eclipse.jetty.client.http.HttpChannelOverHTTP.receive(HttpChannelOverHTTP.java:133)
	at shaded.v9_4.org.eclipse.jetty.client.http.HttpConnectionOverHTTP.onFillable(HttpConnectionOverHTTP.java:151)
	at shaded.v9_4.org.eclipse.jetty.io.AbstractConnection$ReadCallback.succeeded(AbstractConnection.java:311)
	at shaded.v9_4.org.eclipse.jetty.io.FillInterest.fillable(FillInterest.java:103)
	at shaded.v9_4.org.eclipse.jetty.io.ssl.SslConnection$DecryptedEndPoint.onFillable(SslConnection.java:543)
	at shaded.v9_4.org.eclipse.jetty.io.ssl.SslConnection.onFillable(SslConnection.java:398)
	at shaded.v9_4.org.eclipse.jetty.io.ssl.SslConnection$2.succeeded(SslConnection.java:161)
	at shaded.v9_4.org.eclipse.jetty.io.FillInterest.fillable(FillInterest.java:103)
	at shaded.v9_4.org.eclipse.jetty.io.ChannelEndPoint$2.run(ChannelEndPoint.java:117)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.runTask(EatWhatYouKill.java:336)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.doProduce(EatWhatYouKill.java:313)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.tryProduce(EatWhatYouKill.java:171)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.run(EatWhatYouKill.java:129)
	at shaded.v9_4.org.eclipse.jetty.util.thread.ReservedThreadExecutor$ReservedThread.run(ReservedThreadExecutor.java:388)
	at com.databricks.rpc.ShadedInstrumentedQueuedThreadPool$$anon$2.$anonfun$run$6(InstrumentedQueuedThreadPool.scala:136)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.instrumentation.QueuedThreadPoolInstrumenter.trackActiveThreads(QueuedThreadPoolInstrumenter.scala:66)
	at com.databricks.instrumentation.QueuedThreadPoolInstrumenter.trackActiveThreads$(QueuedThreadPoolInstrumenter.scala:63)
	at com.databricks.rpc.ShadedInstrumentedQueuedThreadPool.trackActiveThreads(InstrumentedQueuedThreadPool.scala:102)
	at com.databricks.rpc.ShadedInstrumentedQueuedThreadPool$$anon$2.run(InstrumentedQueuedThreadPool.scala:131)
	at shaded.v9_4.org.eclipse.jetty.util.thread.QueuedThreadPool.runJob(QueuedThreadPool.java:806)
	at shaded.v9_4.org.eclipse.jetty.util.thread.QueuedThreadPool$Runner.run(QueuedThreadPool.java:938)
	at java.base/java.lang.Thread.run(Thread.java:829)

# Test set metrics

In [0]:
y_pred = classifier_random.best_estimator_.predict(X_test)
y_true = Y_test

## Confusion matrices

In [0]:
font = {"family": "normal", "size": 15}

matplotlib.rc("font", **font)
cm = confusion_matrix(y_true, y_pred)
fig, ax = plt.subplots()
sns.heatmap(
    cm,
    annot=True,
    annot_kws={"size": 12},
    ax=ax,
    cmap="Blues",
    fmt="g",
    xticklabels=list(encode_dict.keys()),
    yticklabels=list(encode_dict.keys()),
)
plt.xticks(rotation=45, horizontalalignment="right")
fig.set_size_inches(12, 12)
plt.title("Confusion Matrix - Test Set", fontdict={"family": "normal", "size": 21})
img_buf = io.BytesIO()
plt.ylabel("True label", fontdict={"family": "normal", "size": 21})
plt.xlabel("Predicted label", fontdict={"family": "normal", "size": 21})
plt.show()
fig.savefig(img_buf, dpi=fig.dpi, bbox_inches="tight")

In [0]:
font = {"family": "normal", "size": 15}

matplotlib.rc("font", **font)
cm = confusion_matrix(y_true, y_pred)
fig, ax = plt.subplots()
sns.heatmap(
    cm / cm.sum(axis=1).reshape(-1, 1),
    annot=True,
    annot_kws={"size": 12},
    ax=ax,
    cmap="Blues",
    fmt=".2%",
    xticklabels=list(encode_dict.keys()),
    yticklabels=list(encode_dict.keys()),
)
plt.xticks(rotation=45, horizontalalignment="right")
fig.set_size_inches(12, 12)
plt.title(
    "Confusion Matrix - Test Set Percentage", fontdict={"family": "normal", "size": 21}
)
plt.ylabel("True label", fontdict={"family": "normal", "size": 21})
plt.xlabel("Predicted label", fontdict={"family": "normal", "size": 21})
img_buf2 = io.BytesIO()
plt.show()
fig.savefig(img_buf2, dpi=fig.dpi, bbox_inches="tight")

## F1, precision, recall

In [0]:
how = ["micro", "macro", "weighted"]
metrics = {k: {"f1": 0, "precision": 0, "recall": 0} for k in how}
for k in how:
    metrics[k]["f1"] = f1_score(y_true, y_pred, average=k)
    metrics[k]["precision"] = precision_score(y_true, y_pred, average=k)
    metrics[k]["recall"] = recall_score(y_true, y_pred, average=k)
# now print it nicely
print("Metrics for test set!")
print("+" * 29)
for k in how:
    print(f"{k.upper()} average:")
    for k2 in metrics[k]:
        print(f"\t{k2}: {metrics[k][k2]}")
    print("-" * 29)

# Log confusion matrix + metrics

In [0]:
im = Image.open(img_buf)
mlflow.log_image(im, "confusion_matrix.png")
img_buf.close()
# Now do percentage matrix
im = Image.open(img_buf2)
mlflow.log_image(im, "confusion_matrix_percentage.png")
img_buf.close()

In [0]:
# log f1,prec,recall for micro, macro, and weighted (so 9 total)
for k in how:
    for k2 in metrics[k]:
        mlflow.log_metric(f"{k}_{k2}", metrics[k][k2])

# Log model to run

In [0]:
log_estimator_model(estimator, encode_dict)

# End run

In [0]:
mlflow.end_run()